# Solving the Hamiltonian

In [8]:
using LinearAlgebra

In [5]:
# make the hamiltonian

function random_hamiltonian(dim)
    random_matrix = rand(dim, dim) + im * rand(dim, dim)
    return (random_matrix + random_matrix') / 2  # Ensure the matrix is Herm
end

hamiltonian = random_hamiltonian(4)

4×4 Matrix{ComplexF64}:
 0.804132+0.0im       0.235296+0.123578im  …  0.528974+0.244714im
 0.235296-0.123578im  0.971987+0.0im           0.59018+0.162696im
 0.610909-0.173746im  0.302354-0.114155im     0.167346+0.0734461im
 0.528974-0.244714im   0.59018-0.162696im     0.768502+0.0im

In [7]:
function diagonalise(H)
    return eigen(H).values
end

values = diagonalise(hamiltonian)
println("Eigenvalues: ", values)

Eigenvalues: [-0.05971914298369409, 0.4867688850422934, 0.9000874155435482, 2.16278279236245]


# Mathematical Background: The XYZ Hamiltonian

## What is the XYZ Model?
The XYZ Hamiltonian describes a chain of quantum spins with nearest-neighbor interactions:

$$H = - \frac{1}{2}\sum_{i=1}^{N-1} \left[ J_x \sigma_i^x \sigma_{i+1}^x + J_y \sigma_i^y \sigma_{i+1}^y + J_z \sigma_i^z \sigma_{i+1}^z \right] - \frac{1}{2} h \sum_{i=1}^N \sigma_i^z$$


In [ ]:
function build_xyz_hamiltonian(N::Int, Jx=1.0, Jy=1.0, Jz=1.0, h=0.0)
    """
    Build XYZ Hamiltonian.
    """
    # Define Pauli matrices
    sigma_x = [0 1; 1 0]
    sigma_y = [0 -im; im 0]
    sigma_z = [1 0; 0 -1]
    I2 = [1 0; 0 1]

    # Initialize Hamiltonian matrix
    hamiltonian = zeros(ComplexF64, 2^N, 2^N)

    # Add nearest-neighbor interaction terms
    for i in 1:N
        # Next neighbor with periodic boundary conditions
        j = (i % N) + 1
        
        # Build σᵢˣσⱼˣ term
        local_term_xx = 1
        for site in 1:N
            if site == i || site == j
                local_term_xx = kron(local_term_xx, sigma_x)
            else
                local_term_xx = kron(local_term_xx, I2)
            end
        end
        
        # Build σᵢʸσⱼʸ term
        local_term_yy = 1
        for site in 1:N
            if site == i || site == j
                local_term_yy = kron(local_term_yy, sigma_y)
            else
                local_term_yy = kron(local_term_yy, I2)
            end
        end
        
        # Build σᵢᶻσⱼᶻ term
        local_term_zz = 1
        for site in 1:N
            if site == i || site == j
                local_term_zz = kron(local_term_zz, sigma_z)
            else
                local_term_zz = kron(local_term_zz, I2)
            end
        end
        
        # Add interaction terms (negative sign from Hamiltonian definition)
        hamiltonian -= Jx * local_term_xx + Jy * local_term_yy + Jz * local_term_zz
    end
    
    # Add magnetic field term: -h∑ᵢ σᵢᶻ
    if h != 0.0
        for i in 1:N
            local_term_z = 1
            for site in 1:N
                if site == i
                    local_term_z = kron(local_term_z, sigma_z)
                else
                    local_term_z = kron(local_term_z, I2)
                end
            end
            hamiltonian -= h * local_term_z
        end
    end
    
    # Divide by 2 to avoid double counting (each bond counted twice in periodic BC)
    return hamiltonian / 2
end

build_xyz_hamiltonian (generic function with 5 methods)

In [15]:
N = 3
Jx_val = 1.0
Jy_val = 1.0
Jz_val = 0.5
h_val = 0.1

# Build the Hamiltonian for the specified parameters
hamiltonian_xyz = build_xyz_hamiltonian(N, Jx_val, Jy_val, Jz_val, h_val)

println("XYZ Hamiltonian for N=$N:")
println("Size: ", size(hamiltonian_xyz))
println("Is Hermitian? ", ishermitian(hamiltonian_xyz))
println("-"^50)

# Diagonalize the Hamiltonian using the existing function
eigenvalues = diagonalise(hamiltonian_xyz)

# Print the eigenvalues
println("Eigenvalues (real parts):")
for (i, val) in enumerate(real.(eigenvalues))
    println("E$i = ", round(val, digits=6))
end

println("\nGround state energy: ", minimum(real.(eigenvalues)))
println("Energy gap: ", sort(real.(eigenvalues))[2] - minimum(real.(eigenvalues)))

XYZ Hamiltonian for N=3:
Size: (8, 8)
Is Hermitian? true
--------------------------------------------------
Eigenvalues (real parts):
E1 = -1.8
E2 = -1.7
E3 = -0.9
E4 = -0.6
E5 = 1.2
E6 = 1.2
E7 = 1.3
E8 = 1.3

Ground state energy: -1.7999999999999985
Energy gap: 0.10000000000000009
Size: (8, 8)
Is Hermitian? true
--------------------------------------------------
Eigenvalues (real parts):
E1 = -1.8
E2 = -1.7
E3 = -0.9
E4 = -0.6
E5 = 1.2
E6 = 1.2
E7 = 1.3
E8 = 1.3

Ground state energy: -1.7999999999999985
Energy gap: 0.10000000000000009
